**Conference Notebook Kernel: `bw25`**

# Using correlated time-series from `ENTSO-E transparency platform`

This data has been preprocessed using code from [akula](https://github.com/aleksandra-kim/akula/). We will work directly with datapackages.

In [ ]:
from fs.zipfs import ZipFS
from tqdm import tqdm
import pandas as pd
import bw_processing as bwp
import bw2calc as bc
import seaborn as sb
import numpy as np

Our functional unit is 1 kWh of low-voltage electricity from Switzerland

In [ ]:
fu = {5204: 1}

The datapackages are already prepared on the server

In [ ]:
data_objs = [
    bwp.load_datapackage(ZipFS("/srv/data/entso/biosphere3.5d405d71.zip")),
    bwp.load_datapackage(ZipFS("/srv/data/entso/ecoinvent-38-cutoff.676d1727.zip")),
    bwp.load_datapackage(ZipFS("/srv/data/entso/Swiss-residual-electricity-mix.1c9d19b3.zip")),
    bwp.load_datapackage(ZipFS("/srv/data/entso/ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf.zip")),
    bwp.load_datapackage(ZipFS("/srv/data/entso/entso-timeseries-2021.zip")),
]

First, make calculations without the time-series data:

In [ ]:
lca = bc.LCA(
    demand=fu,
    data_objs=data_objs,
    use_distributions=True,
    use_arrays=False
)
lca.lci()
lca.lcia()

no_corr = np.array([lca.score for _ in zip(range(20), lca)])

Then use the ENTSO data as well (`use_arrays=True`)

In [ ]:
lca = bc.LCA(
    demand=fu,
    data_objs=data_objs,
    use_distributions=True,
    use_arrays=True
)
lca.lci()
lca.lcia()

with_corr = np.array([lca.score for _ in zip(range(20), lca)])

The differences are dignificant:

In [ ]:
sb.histplot(pd.DataFrame({'Uncorrelated': no_corr, 'Time series': with_corr}), kde=True, bins=5)

## Next steps

* Could only take data from day or night - time series data is hourly and starts at midnight 2021, need custom indexer
* Could differentiate by season - custom indexer
* Could look at 2022 - need to get ENTSO data